This code is designed to utilize the Sportsipy API to consolidate nfl player stats from the 2020 season.  The output is a csv file with player stats organized in a weekly basis.

In [2]:
# Import dependencies
import pandas as pd
import csv
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import classes
from sportsipy.nfl.teams import Teams
from sportsipy.nfl.schedule import Schedule
from sportsipy.nfl.boxscore import Boxscore

In [4]:
# Iterate through nfl teams in the 2020 season and output each team abbreviation
team_abv = []
for team in Teams(year = 2020).dataframes.index:
      team_abv.append(team)

In [5]:
team_abv

['GNB',
 'BUF',
 'TAM',
 'OTI',
 'NOR',
 'KAN',
 'RAV',
 'SEA',
 'CLT',
 'RAI',
 'MIN',
 'PIT',
 'CRD',
 'CLE',
 'MIA',
 'ATL',
 'DAL',
 'HTX',
 'SDG',
 'DET',
 'SFO',
 'RAM',
 'CHI',
 'CAR',
 'WAS',
 'PHI',
 'NWE',
 'DEN',
 'CIN',
 'JAX',
 'NYG',
 'NYJ']

In [6]:
# Iterate through the nfl 2020 schedule using team_abv as the range to output a list of a list containing every game
team_schedule = []
for i in range(len(team_abv)):
    schedule = list(Schedule(team_abv[i], year = '2020'))
    team_schedule.append(schedule)

In [7]:
team_schedule

[[September 13 - MIN,
  September 20 - DET,
  September 27 - NOR,
  October 5 - ATL,
  October 18 - TAM,
  October 25 - HTX,
  November 1 - MIN,
  November 5 - SFO,
  November 15 - JAX,
  November 22 - CLT,
  November 29 - CHI,
  December 6 - PHI,
  December 13 - DET,
  December 19 - CAR,
  December 27 - OTI,
  January 3 - CHI,
  January 16 - RAM,
  January 24 - TAM],
 [September 13 - NYJ,
  September 20 - MIA,
  September 27 - RAM,
  October 4 - RAI,
  October 13 - OTI,
  October 19 - KAN,
  October 25 - NYJ,
  November 1 - NWE,
  November 8 - SEA,
  November 15 - CRD,
  November 29 - SDG,
  December 7 - SFO,
  December 13 - PIT,
  December 19 - DEN,
  December 28 - NWE,
  January 3 - MIA,
  January 9 - CLT,
  January 16 - RAV,
  January 24 - KAN],
 [September 13 - NOR,
  September 20 - CAR,
  September 27 - DEN,
  October 4 - SDG,
  October 8 - CHI,
  October 18 - GNB,
  October 25 - RAI,
  November 2 - NYG,
  November 8 - NOR,
  November 15 - CAR,
  November 23 - RAM,
  November 29 

In [8]:
# Limit schedule list to the regular season (first 16 games)
regular_season = []
for s in team_schedule:
    regular_season.append(s[0:16])

In [9]:
regular_season

[[September 13 - MIN,
  September 20 - DET,
  September 27 - NOR,
  October 5 - ATL,
  October 18 - TAM,
  October 25 - HTX,
  November 1 - MIN,
  November 5 - SFO,
  November 15 - JAX,
  November 22 - CLT,
  November 29 - CHI,
  December 6 - PHI,
  December 13 - DET,
  December 19 - CAR,
  December 27 - OTI,
  January 3 - CHI],
 [September 13 - NYJ,
  September 20 - MIA,
  September 27 - RAM,
  October 4 - RAI,
  October 13 - OTI,
  October 19 - KAN,
  October 25 - NYJ,
  November 1 - NWE,
  November 8 - SEA,
  November 15 - CRD,
  November 29 - SDG,
  December 7 - SFO,
  December 13 - PIT,
  December 19 - DEN,
  December 28 - NWE,
  January 3 - MIA],
 [September 13 - NOR,
  September 20 - CAR,
  September 27 - DEN,
  October 4 - SDG,
  October 8 - CHI,
  October 18 - GNB,
  October 25 - RAI,
  November 2 - NYG,
  November 8 - NOR,
  November 15 - CAR,
  November 23 - RAM,
  November 29 - KAN,
  December 13 - MIN,
  December 20 - ATL,
  December 26 - DET,
  January 3 - ATL],
 [Septemb

In [10]:
# Iterate through each team schedule in the regular season and output each boxscore URI for every game
boxscore_index_list = []
for team_sch in regular_season:
#     print(team_sch)
    for game in team_sch:
#         print(game.boxscore_index)  # Prints the boxscore URI for each game in a team's schedule
        boxscore = game.boxscore_index  # Returns an instance of the Boxscore class for this specific game
        boxscore_index_list.append(boxscore)

In [11]:
# 32 teams x 16 games = 512 boxscore URIs
len(boxscore_index_list)

512

In [12]:
# Iterate through every boxscore URI to output each boxscore to a list
game_data_list=[]
for i in range(len(boxscore_index_list)):
    game_data = Boxscore(boxscore_index_list[i])
#     print(game_data)
    game_data_list.append(game_data)

In [13]:
len(game_data_list)

512

In [14]:
# Function that utilizes the Boxscore class properties ‘home_players’ and ‘away_players’ 
# to concatenate each individual BoxscorePlayer dataframe
def home_away_split(game_data):
#     Home
    home_df = game_data.home_players[0].dataframe
    for player in game_data.home_players[1:]:
        home_df = pd.concat([home_df, player.dataframe], axis = 0)
    home_df['name'] = [x.name for x in game_data.home_players]
    home_df.set_index('name', inplace = True)
    
#     Away
    away_df = game_data.away_players[0].dataframe
    for player in game_data.away_players[1:]:
        away_df = pd.concat([away_df, player.dataframe], axis = 0)
    away_df['name'] = [x.name for x in game_data.away_players]
    away_df.set_index('name', inplace = True)

    return home_df, away_df

In [15]:
# Append boxscores for all games
full_df_home = []
full_df_away = []

for i in range(len(game_data_list)) :       
    first_df, second_df = home_away_split(game_data_list[i])

    full_df_home.append(first_df)
    full_df_away.append(second_df)

In [16]:
# Concatnate all home boxscores
homebox_df = pd.concat(full_df_home)
homebox_df.head(10)

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,punt_return_touchdown,longest_punt_return,extra_points_made,extra_points_attempted,field_goals_made,field_goals_attempted,punts,total_punt_yards,yards_per_punt,longest_punt
name,,,,,,,,,,,,,,,,,,,,,
Kirk Cousins,19,25,259,2,1,2,11,37,118.6,4,...,None,None,None,None,None,None,None,None,None,None
Alexander Mattison,0,0,0,0,0,0,0,0,NaN,6,...,None,None,None,None,None,None,None,None,None,None
Dalvin Cook,0,0,0,0,0,0,0,0,NaN,12,...,None,None,None,None,None,None,None,None,None,None
Adam Thielen,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Olabisi Johnson,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Kyle Rudolph,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Justin Jefferson,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Irv Smith Jr.,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Tajae Sharpe,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None


In [17]:
len(homebox_df)

15880

In [18]:
# Concatnate all away boxscores
awaybox_df = pd.concat(full_df_away)
awaybox_df.head(10)

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,punt_return_touchdown,longest_punt_return,extra_points_made,extra_points_attempted,field_goals_made,field_goals_attempted,punts,total_punt_yards,yards_per_punt,longest_punt
name,,,,,,,,,,,,,,,,,,,,,
Aaron Rodgers,32,44,364,4,0,0,0,45,127.5,1,...,None,None,None,None,None,None,None,None,None,None
Aaron Jones,0,0,0,0,0,0,0,0,NaN,16,...,None,None,None,None,None,None,None,None,None,None
Tyler Ervin,0,0,0,0,0,0,0,0,NaN,3,...,0,0,None,None,None,None,None,None,None,None
Jamaal Williams,0,0,0,0,0,0,0,0,NaN,7,...,None,None,None,None,None,None,None,None,None,None
Allen Lazard,0,0,0,0,0,0,0,0,NaN,1,...,None,None,None,None,None,None,None,None,None,None
AJ Dillon,0,0,0,0,0,0,0,0,NaN,2,...,None,None,None,None,None,None,None,None,None,None
Davante Adams,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Marquez Valdes-Scantling,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Josiah Deguara,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None


In [19]:
len(awaybox_df)

15864

In [20]:
# Concatnate all games for 2020 season
frames = [homebox_df, awaybox_df]
season_df = pd.concat(frames)

In [21]:
len(season_df)

31744

In [22]:
season_df.head(10)

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,punt_return_touchdown,longest_punt_return,extra_points_made,extra_points_attempted,field_goals_made,field_goals_attempted,punts,total_punt_yards,yards_per_punt,longest_punt
name,,,,,,,,,,,,,,,,,,,,,
Kirk Cousins,19,25,259,2,1,2,11,37,118.6,4,...,None,None,None,None,None,None,None,None,None,None
Alexander Mattison,0,0,0,0,0,0,0,0,NaN,6,...,None,None,None,None,None,None,None,None,None,None
Dalvin Cook,0,0,0,0,0,0,0,0,NaN,12,...,None,None,None,None,None,None,None,None,None,None
Adam Thielen,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Olabisi Johnson,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Kyle Rudolph,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Justin Jefferson,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Irv Smith Jr.,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Tajae Sharpe,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None


In [23]:
# Replace missing values with 0
season_df = season_df.fillna(0)

In [24]:
# Sum of missing values
season_df.isnull().sum()

completed_passes                        0
attempted_passes                        0
passing_yards                           0
passing_touchdowns                      0
interceptions_thrown                    0
times_sacked                            0
yards_lost_from_sacks                   0
longest_pass                            0
quarterback_rating                      0
rush_attempts                           0
rush_yards                              0
rush_touchdowns                         0
longest_rush                            0
times_pass_target                       0
receptions                              0
receiving_yards                         0
receiving_touchdowns                    0
longest_reception                       0
fumbles                                 0
fumbles_lost                            0
interceptions                           0
yards_returned_from_interception        0
interceptions_returned_for_touchdown    0
longest_interception_return       

In [25]:
season_df.head(10)

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,punt_return_touchdown,longest_punt_return,extra_points_made,extra_points_attempted,field_goals_made,field_goals_attempted,punts,total_punt_yards,yards_per_punt,longest_punt
name,,,,,,,,,,,,,,,,,,,,,
Kirk Cousins,19,25,259,2,1,2,11,37,118.6,4,...,0,0,0,0,0,0,0,0,0.0,0
Alexander Mattison,0,0,0,0,0,0,0,0,0.0,6,...,0,0,0,0,0,0,0,0,0.0,0
Dalvin Cook,0,0,0,0,0,0,0,0,0.0,12,...,0,0,0,0,0,0,0,0,0.0,0
Adam Thielen,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0.0,0
Olabisi Johnson,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0.0,0
Kyle Rudolph,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0.0,0
Justin Jefferson,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0.0,0
Irv Smith Jr.,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0.0,0
Tajae Sharpe,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0.0,0


In [26]:
# Count duplicated rows (for some reason code is duplicating player stats for each game)
season_df.duplicated().sum()

24929

In [27]:
# Drop duplicates - this also drops any player stats for a game in which they did not accrue any stats
season2020_df= season_df.drop_duplicates()

In [28]:
# 32 teams x ~16 games equals 512 games.  6815 rows equates to ~13 players accruing stats per game (reasonable)
len(season2020_df)

6815

In [29]:
# Drop columns that don't accure fantasy stats
season2020_df = season2020_df.drop(['interceptions',
                'yards_returned_from_interception',
                'interceptions_returned_for_touchdown',
                'longest_interception_return',
                'passes_defended',
                'sacks',
                'combined_tackles',
                'solo_tackles',
                'assists_on_tackles',
                'tackles_for_loss',
                'quarterback_hits',
                'fumbles_recovered',
                'yards_recovered_from_fumble',
                'fumbles_forced',
                'kickoff_returns',
                'kickoff_return_yards',
                'average_kickoff_return_yards',
                'longest_kickoff_return',
                'punt_returns',
                'punt_return_yards',
                'yards_per_punt_return',
                'longest_punt_return',
                'extra_points_made',
                'extra_points_attempted',
                'field_goals_made',
                'field_goals_attempted',
                'punts',
                'total_punt_yards',
                'yards_per_punt',
                'longest_punt'
                ], axis=1)

In [30]:
season2020_df

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,times_pass_target,receptions,receiving_yards,receiving_touchdowns,longest_reception,fumbles,fumbles_lost,fumbles_recovered_for_touchdown,kickoff_return_touchdown,punt_return_touchdown
name,,,,,,,,,,,,,,,,,,,,,
Kirk Cousins,19,25,259,2,1,2,11,37,118.6,4,...,0,0,0,0,0,0,0,0,0,0
Alexander Mattison,0,0,0,0,0,0,0,0,0.0,6,...,4,4,30,0,9,0,0,0,0,0
Dalvin Cook,0,0,0,0,0,0,0,0,0.0,12,...,2,1,-2,0,-2,0,0,0,0,0
Adam Thielen,0,0,0,0,0,0,0,0,0.0,0,...,8,6,110,2,37,0,0,0,0,0
Olabisi Johnson,0,0,0,0,0,0,0,0,0.0,0,...,4,3,56,0,29,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Evan Engram,0,0,0,0,0,0,0,0,0.0,0,...,9,6,129,0,53,1,1,0,0,0
Leonard Williams,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
Jabrill Peppers,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


Below code is desinged to web scrape the position for each player from https://www.pro-football-reference.com/years/2020/fantasy.htm.  This is the same data the Sportsipy API utilizes.

In [31]:
# Import dependencies
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [32]:
# Creating the BeautifulSoup object
year = 2020
url = "https://www.pro-football-reference.com/years/{}/fantasy.htm".format(year) # String formatting to manipulate the url by a given year
html = urlopen(url)
soup = BeautifulSoup(html)

In [33]:
# Getting the table row data - two lists (player and position)

# Grab all rows that are not classed as table header rows - football reference places a table header row every 30 rows 
rows = soup.findAll('tr', class_ = lambda table_rows: table_rows != "thead") 


player = [[td.getText() for td in rows[i].find_all('td', attrs={'data-stat':'player'})]
                for i in range(len(rows))]
player = player[2:] # Removes the first two empty rows from the player stats list

position = [[td.getText() for td in rows[i].find_all('td', attrs={'data-stat': 'fantasy_pos'})]
                for i in range(len(rows))]
position = position[2:] # Removes the first two empty rows from the player stats list

In [34]:
# Reduce player from a list of a list to a list
out1 = []
for sublist in player:
    out1.extend(sublist)

In [35]:
out1

['Derrick Henry *+',
 'Alvin Kamara*',
 'Dalvin Cook*',
 'Travis Kelce*+',
 'Davante Adams*+',
 'Tyreek Hill*+',
 'Josh Allen*',
 'Aaron Rodgers*+',
 'Kyler Murray*',
 'Patrick Mahomes*',
 'Deshaun Watson*',
 'Jonathan Taylor',
 'Aaron Jones*',
 'Darren Waller*',
 'David Montgomery',
 'Russell Wilson*',
 'Stefon Diggs*+',
 'James Robinson',
 'Josh Jacobs*',
 'Calvin Ridley',
 'D.K. Metcalf*',
 'Nick Chubb*',
 'Ryan Tannehill',
 'Justin Jefferson*',
 'Tom Brady',
 'Adam Thielen',
 'Mike Evans',
 'A.J. Brown*',
 'Justin Herbert',
 'Lamar Jackson',
 'Kareem Hunt',
 'DeAndre Hopkins*',
 'Ezekiel Elliott',
 'Tyler Lockett',
 'Kenyan Drake',
 'Melvin Gordon',
 'Antonio Gibson ',
 'Allen Robinson',
 'Robert Tonyan',
 'Robert Woods',
 'Ronald Jones II',
 'Chase Claypool',
 'Marvin Jones',
 'Brandin Cooks',
 'Kirk Cousins',
 'Taysom Hill',
 'Mark Andrews',
 'Chris Carson',
 'J.K. Dobbins',
 'D.J. Moore',
 'Keenan Allen*',
 'Amari Cooper',
 'T.J. Hockenson*',
 'Mike Davis',
 'CeeDee Lamb',
 'Dav

In [36]:
# Reduce position from a list of a list to a list
out2 = []
for sublist in position:
    out2.extend(sublist)

In [37]:
out2

['RB',
 'RB',
 'RB',
 'TE',
 'WR',
 'WR',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'RB',
 'RB',
 'TE',
 'RB',
 'QB',
 'WR',
 'RB',
 'RB',
 'WR',
 'WR',
 'RB',
 'QB',
 'WR',
 'QB',
 'WR',
 'WR',
 'WR',
 'QB',
 'QB',
 'RB',
 'WR',
 'RB',
 'WR',
 'RB',
 'RB',
 'RB',
 'WR',
 'TE',
 'WR',
 'RB',
 'WR',
 'WR',
 'WR',
 'QB',
 'WR',
 'TE',
 'RB',
 'RB',
 'WR',
 'WR',
 'WR',
 'TE',
 'RB',
 'WR',
 'RB',
 'TE',
 'TE',
 'TE',
 'RB',
 'RB',
 'WR',
 'WR',
 'WR',
 'RB',
 'WR',
 'TE',
 'WR',
 'RB',
 'WR',
 'TE',
 'TE',
 'WR',
 'WR',
 'QB',
 '',
 'RB',
 'WR',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'QB',
 'RB',
 'RB',
 'WR',
 'WR',
 'WR',
 'WR',
 'RB',
 'RB',
 'WR',
 'RB',
 'WR',
 'QB',
 'RB',
 'WR',
 'WR',
 'RB',
 'WR',
 'RB',
 'RB',
 'RB',
 'RB',
 'RB',
 'WR',
 'WR',
 'WR',
 'QB',
 'WR',
 'RB',
 'WR',
 'QB',
 'WR',
 'WR',
 'WR',
 'WR',
 'WR',
 'WR',
 'WR',
 'WR',
 'WR',
 'RB',
 'RB',
 'WR',
 'RB',
 'WR',
 'RB',
 

In [38]:
# Creating a Pandas DataFrame object from the two lists
d = {'name':out1, 'position': out2}
position_df = pd.DataFrame(d, columns=['name','position'])
position_df

# Removing symbols from names
symbols = ["*", "+"]
for name in symbols:
    position_df["name"] = position_df["name"].str.replace(name,"")
    
position_df.set_index('name', inplace = True)

position_df

,position
name,
Derrick Henry,RB
Alvin Kamara,RB
Dalvin Cook,RB
Travis Kelce,TE
Davante Adams,WR
...,...
Dwayne Harris,
Nate Sudfeld,QB
Nsimba Webster,


In [39]:
# Merge position data into player stat data
nfl_2020_df = season2020_df.merge(position_df, how='left', on=["name"])

# Any player that isn't a QB, RB, WR, TE will be assigned as a D
nfl_2020_df.replace('', np.nan, inplace=True)
nfl_2020_df = nfl_2020_df.fillna('D')

nfl_2020_df

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,receptions,receiving_yards,receiving_touchdowns,longest_reception,fumbles,fumbles_lost,fumbles_recovered_for_touchdown,kickoff_return_touchdown,punt_return_touchdown,position
name,,,,,,,,,,,,,,,,,,,,,
Kirk Cousins,19,25,259,2,1,2,11,37,118.6,4,...,0,0,0,0,0,0,0,0,0,QB
Alexander Mattison,0,0,0,0,0,0,0,0,0.0,6,...,4,30,0,9,0,0,0,0,0,RB
Dalvin Cook,0,0,0,0,0,0,0,0,0.0,12,...,1,-2,0,-2,0,0,0,0,0,RB
Adam Thielen,0,0,0,0,0,0,0,0,0.0,0,...,6,110,2,37,0,0,0,0,0,WR
Olabisi Johnson,0,0,0,0,0,0,0,0,0.0,0,...,3,56,0,29,0,0,0,0,0,WR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Evan Engram,0,0,0,0,0,0,0,0,0.0,0,...,6,129,0,53,1,1,0,0,0,TE
Leonard Williams,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,D
Jabrill Peppers,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,D


In [40]:
# Fantasy points scored function (excluding 2pt conv)

nfl_2020_df['fantasy_points'] = (nfl_2020_df['passing_touchdowns']*4 + nfl_2020_df['passing_yards']*.04
+ nfl_2020_df['rush_yards']*.1 + nfl_2020_df['rush_touchdowns']*6 + nfl_2020_df['receptions']
+ nfl_2020_df['receiving_yards']*.1 + nfl_2020_df['receiving_touchdowns']*6 + nfl_2020_df['punt_return_touchdown']*6 
+ nfl_2020_df['fumbles_recovered_for_touchdown']*6 + nfl_2020_df['kickoff_return_touchdown']*6
- nfl_2020_df['fumbles_lost'] - nfl_2020_df['interceptions_thrown'] 
+ nfl_2020_df['passing_yards'].apply(lambda x :3 if x >= 300 else 0)
+ nfl_2020_df['rush_yards'].apply(lambda x :3 if x >= 100 else 0)
+ nfl_2020_df['receiving_yards'].apply(lambda x :3 if x >= 100 else 0))

In [41]:
nfl_2020_df

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,receiving_yards,receiving_touchdowns,longest_reception,fumbles,fumbles_lost,fumbles_recovered_for_touchdown,kickoff_return_touchdown,punt_return_touchdown,position,fantasy_points
name,,,,,,,,,,,,,,,,,,,,,
Kirk Cousins,19,25,259,2,1,2,11,37,118.6,4,...,0,0,0,0,0,0,0,0,QB,20.76
Alexander Mattison,0,0,0,0,0,0,0,0,0.0,6,...,30,0,9,0,0,0,0,0,RB,12.00
Dalvin Cook,0,0,0,0,0,0,0,0,0.0,12,...,-2,0,-2,0,0,0,0,0,RB,17.80
Adam Thielen,0,0,0,0,0,0,0,0,0.0,0,...,110,2,37,0,0,0,0,0,WR,32.00
Olabisi Johnson,0,0,0,0,0,0,0,0,0.0,0,...,56,0,29,0,0,0,0,0,WR,8.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Evan Engram,0,0,0,0,0,0,0,0,0.0,0,...,129,0,53,1,1,0,0,0,TE,20.90
Leonard Williams,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,D,0.00
Jabrill Peppers,0,0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,D,0.00


In [42]:
# Count number of defensive players
(nfl_2020_df.position == 'D').sum()

2457

In [43]:
# Drop rows with defensive players
nfl_2020_df = nfl_2020_df[nfl_2020_df.position != 'D']

In [44]:
nfl_2020_df

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,receiving_yards,receiving_touchdowns,longest_reception,fumbles,fumbles_lost,fumbles_recovered_for_touchdown,kickoff_return_touchdown,punt_return_touchdown,position,fantasy_points
name,,,,,,,,,,,,,,,,,,,,,
Kirk Cousins,19,25,259,2,1,2,11,37,118.6,4,...,0,0,0,0,0,0,0,0,QB,20.76
Alexander Mattison,0,0,0,0,0,0,0,0,0.0,6,...,30,0,9,0,0,0,0,0,RB,12.00
Dalvin Cook,0,0,0,0,0,0,0,0,0.0,12,...,-2,0,-2,0,0,0,0,0,RB,17.80
Adam Thielen,0,0,0,0,0,0,0,0,0.0,0,...,110,2,37,0,0,0,0,0,WR,32.00
Olabisi Johnson,0,0,0,0,0,0,0,0,0.0,0,...,56,0,29,0,0,0,0,0,WR,8.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wayne Gallman,0,0,0,0,0,0,0,0,0.0,24,...,-3,0,4,0,0,0,0,0,RB,18.10
Dion Lewis,0,0,0,0,0,0,0,0,0.0,2,...,7,0,7,0,0,0,0,0,RB,2.70
Alfred Morris,0,0,0,0,0,0,0,0,0.0,4,...,0,0,0,0,0,0,0,0,RB,0.80


In [45]:
# Convert dataframe to csv
nfl_2020_df.to_csv('nfl_2020_player_stats.csv')